# Importar bibliotecas generales

In [1]:
from sys import maxsize #para imprimir arrays completos
import numpy as np
import pandas as pd
import re as regex
import math as math
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de opciones generales

In [2]:
#cantidad de elementos que se muestran cuando se imprimen resultados
np.set_printoptions(threshold=maxsize)

pd.set_option('display.max_columns',1000000)

# Importar bibliotecas propias 

In [3]:
#biblioteca para completar valores Nan desde las columnas description y title
%run "analize_description_title.ipynb"

# Leer el data set desde el archivo y generar el data frame

In [4]:
pathArchivoDataSet = 'properatti.csv'
df = pd.read_csv(pathArchivoDataSet)

# Definimos datos generales que se van a usar en la notebook

In [5]:
#definimos un diccionario con valores de conversion entre monedas locales y el USD
conversion_USD_a_monedas_locales = { 'ARS': 63, 'PEN': 3.53, 'UYU': 43.41, 'USD': 1}

# Detalle generales del data frame

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 no

In [7]:
df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,USD,1676227.5,95000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,USD,1129248.0,64000.0,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [8]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# La primer columna no tiene un nombre asignado

In [9]:
type(df.iloc[0,0])

numpy.int64

In [10]:
len(df.loc[df.index == df['Unnamed: 0']])

121220

## Para todas las filas del data frame, el valor de la primer columna es igual al valor del index. Se puede asumir que dicha columna corresponde a un campo ID. Ya que contamos con el index del data frame, borramos la columna ID

In [11]:
df.rename(columns={'Unnamed: 0': 'Id'}, inplace = True)
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

# Limpieza del data set 

### Removemos filas duplicadas

In [12]:
print('Total de filas del data frame: ' + str(len(df)))

Total de filas del data frame: 121220


In [13]:
#eliminamos duplicados verificando las siguientes columnas
df.drop_duplicates(subset=['operation', 'property_type','place_name','place_with_parent_names', \
                          'country_name', 'state_name', 'lat', 'lon', 'price', 'currency', 'price_aprox_local_currency', \
                          'price_aprox_usd','surface_total_in_m2', 'surface_covered_in_m2',\
                          'price_usd_per_m2','price_per_m2','floor','expenses', 'description', 'title'],keep='last', inplace=True)

In [14]:
print('Total de filas del data frame sin duplicados: ' + str(len(df)))

Total de filas del data frame sin duplicados: 116068


In [15]:
#reseteamos los índices para poder seguir usándolos sin problemas
df = df.reset_index(drop=True);

### Se eliminan columnas innecesarias

In [16]:
df[~df['floor'].isnull() & df['floor'] > 20]

,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail


In [17]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df = df.drop(labels='floor', axis = 1)

In [18]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df = df.drop(labels='expenses', axis = 1)

In [19]:
#quitamos la columna image_thumbnail ya que no aporta información relevante
df = df.drop(labels='image_thumbnail', axis= 1)

In [20]:
#quitamos la columna properati_url ya que no aporta información relevante
df = df.drop(labels='properati_url', axis= 1)

In [21]:
#chequeamos las columnas que nos quedan
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'description', 'title'],
      dtype='object')

### Se eliminan datos (filas) que no pudieron ser completados

In [22]:
#se eliminan filas cuyo campo description está vacío, dicha columna se tomó como fuente para otras columnas
#al venir vacía otros datos no pueden ser completados
emptyDescriptionIndexes = df[df['description'].isnull()].index

#borramos las filas con los correspondientes indices
df = df.drop(emptyDescriptionIndexes)

#reseteamos los índices para poder seguir usándolos sin problemas
df = df.reset_index(drop=True);

# Completar valores faltantes

### A partir del dato geonames_id, generamos tres columnas nuevas, que corresponden a la coordenada representativa de cada zona (representada por un geonames_id). Esta coordenada no corresponde a la de la propiedad

##### Leemos directamente el archivo CSV generado en la notebook auxiliar "Completar coordenadas desde geonames_id" . Para ver el proceso remitirse a dicha notebook

In [23]:
latLngCSVFileName = 'latLngFromGeonames.csv'
#cargamos el archivo
latLongDF = pd.read_csv(latLngCSVFileName)

In [24]:
#mergeamos el data frame leído con el da properatti usando el dato geonames_id
df = df.merge(latLongDF, how='left', left_on='geonames_id', right_on='geonames_id', suffixes=('', '_geonames'))

### Datos de localidades: place_with_parent_names, place_name, country_name, state_name

In [25]:
df[['place_with_parent_names', 'place_name','country_name','state_name']].isnull().sum()

place_with_parent_names     0
place_name                 23
country_name                0
state_name                  0
dtype: int64

##### Solo faltan 23 valores en la columna 'place_name' (información de barrio, zona, ciudad, etc.) que se intentarán obtener del la columna 'place_with_parent_names'.

In [26]:
#seteamos la longitud del output para mejor lectura
pd.set_option('display.max_colwidth', -1)

##### Tomamos las columnas necesarias para completar los faltantes

In [27]:
df[['place_with_parent_names', 'place_name','country_name','state_name']]

,place_with_parent_names,place_name,country_name,state_name
0,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
1,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,La Plata,Argentina,Bs.As. G.B.A. Zona Sur
2,|Argentina|Capital Federal|Mataderos|,Mataderos,Argentina,Capital Federal
3,|Argentina|Capital Federal|Liniers|,Liniers,Argentina,Capital Federal
4,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Centro,Argentina,Buenos Aires Costa Atlántica
...,...,...,...,...
116061,|Argentina|Capital Federal|Belgrano|,Belgrano,Argentina,Capital Federal
116062,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|Beccar|,Beccar,Argentina,Bs.As. G.B.A. Zona Norte
116063,|Argentina|Capital Federal|Villa Urquiza|,Villa Urquiza,Argentina,Capital Federal
116064,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Plaza Colón|,Plaza Colón,Argentina,Buenos Aires Costa Atlántica


##### Se crea una nueva columna con los valores de la columna 'place_with_parent_names' en forma de lista

In [28]:
def disgrega(valor):
    return valor.strip('|').split('|')

df['lista_auxiliar'] = df['place_with_parent_names'].apply(lambda x: disgrega(x))

In [29]:
df['lista_auxiliar']

0         [Argentina, Capital Federal, Mataderos]                              
1         [Argentina, Bs.As. G.B.A. Zona Sur, La Plata]                        
2         [Argentina, Capital Federal, Mataderos]                              
3         [Argentina, Capital Federal, Liniers]                                
4         [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Centro]     
                                        ...                                    
116061    [Argentina, Capital Federal, Belgrano]                               
116062    [Argentina, Bs.As. G.B.A. Zona Norte, San Isidro, Beccar]            
116063    [Argentina, Capital Federal, Villa Urquiza]                          
116064    [Argentina, Buenos Aires Costa Atlántica, Mar del Plata, Plaza Colón]
116065    [Argentina, Capital Federal]                                         
Name: lista_auxiliar, Length: 116066, dtype: object

##### Se calcula cuantos elementos tiene cada lista

In [30]:
df['conteo'] = df['lista_auxiliar'].apply(lambda x: len(x))
df['conteo']

0         3
1         3
2         3
3         3
4         4
         ..
116061    3
116062    4
116063    3
116064    4
116065    2
Name: conteo, Length: 116066, dtype: int64

##### Se determina cual es el mayor y menor números de elemntos en un registro de 'place_with_parent_names'

In [31]:
df['conteo'].max()

5

In [32]:
df['conteo'].min()

2

##### Se necesitarán 5 columnas como máximo para desempacar las distintas jerarquías de las locaciones. Mediante el comando apply se obtienen 5 columnas extras con información de localización por regiones.

In [33]:
def chequeo2(valor):
    if len(valor) > 2:
        return valor[2]
    
    
def chequeo3(valor):
    if len(valor) > 3:
        return valor[3]
    
def chequeo4(valor):
    if len(valor) > 4:
        return valor[4]

df['loc1'] = df['lista_auxiliar'].apply(lambda x: x[0])
df['loc2'] = df['lista_auxiliar'].apply(lambda x: x[1])
df['loc3'] = df['lista_auxiliar'].apply(lambda x: chequeo2(x))
df['loc4'] = df['lista_auxiliar'].apply(lambda x: chequeo3(x))
df['loc5'] = df['lista_auxiliar'].apply(lambda x: chequeo4(x))

In [34]:
daux = df[df['loc5'].notnull()]
daux[['loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,loc1,loc2,loc3,loc4,loc5
17,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
33,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio El Golf
269,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
400,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
430,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Las Glorietas
...,...,...,...,...,...
112307,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,QBay Yacht
113366,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Los Lagos
113894,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago
114594,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,Barrio Barrancas del Lago


##### Verificamos que los registros que tenían place_name vacíos tengan valores válidos en las nuevas columnas para cubrir el faltante

In [35]:
len(df[df['place_name'].isna() & ~df['loc5'].isna()])

0

In [36]:
df[df['place_name'].isna()][['lat','lon','geonames_id','country_name', 'state_name','place_name','place_with_parent_names', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

,lat,lon,geonames_id,country_name,state_name,place_name,place_with_parent_names,loc1,loc2,loc3,loc4,loc5
6085,-34.400897,-58.638098,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
9741,-34.400704,-58.638067,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
10958,-34.402688,-58.626261,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
14318,-34.400799,-58.638174,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
18038,-34.403278,-58.626350,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
21286,-34.399704,-58.638405,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
23018,-34.400516,-58.641800,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
24059,-34.400729,-58.637778,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
37930,-34.425087,-58.579659,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None
44891,-34.396074,-58.637380,NaN,Argentina,Bs.As. G.B.A. Zona Norte,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,None,None


##### Vemos que a las fila que les falta el place_name corresponden todas al municipio de Tigre

In [37]:
#completamos los que tienen Nan en place_name con el valor Tigre
df.loc[df['place_name'].isnull(),'place_name'] = 'Tigre'

In [38]:
len(df[df['place_name'].isnull()])

0

##### Eliminamos columnas auxiliares generadas

In [39]:
df.drop(labels=['loc1', 'loc2','loc3','loc4','loc5', 'conteo','lista_auxiliar'], axis = 1, inplace = True)

##### Estandarizaramos los valores de localizaciones

In [40]:
#los datos de países ya están estandarizados
list(df['country_name'].unique())

['Argentina']

In [41]:
#hay registros que tienen en place_name nombres de zonas del conourbano bonaerense
df[df['place_name'] == 'Bs.As. G.B.A. Zona Norte']

,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,description,title,lat_geonames,lat-lon_geonames,lon_geonames,place_name_geonames
1734,1856,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500.0,70.0,NaN,NaN,NaN,"Casa 500m², Argentina, Bs.As. G.B.A. Zona Norte, por U$S 50.000",U$D 50.000 - Casa en Venta - S/n S/N,-36.0,"-36,-60",-60.0,Buenos Aires
2176,2448,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,150.0,NaN,NaN,3.0,"CASA ESTILO ALPINA, DE TRES PLANTAS EN BARRIO LOS CARDALES COUNTRY CLUB . COCINA LIVING COMEDOR, TRES DORMITORIOS DOS BAÑOS . COCHERA. GALERIA PARRILLA LAVADERO. CASA SOBREUN SOLO LOTE DE 750 MTS EXPENSAS $ 7500 PESOS.",U$D 175.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
3559,3919,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750.0,150.0,NaN,NaN,3.0,"CASA ESTILO ALPINA, DE TRES PLANTAS EN BARRIO LOS CARDALES COUNTRY CLUB . COCINA LIVING COMEDOR, TRES DORMITORIOS DOS BAÑOS . COCHERA. GALERIA PARRILLA LAVADERO. CASA SOBRE DOS LOTES DE 750 MTS CADA UNO.EXPENSAS $ 14000 PESOS.",U$D 190.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
3877,4239,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2400.0,170.0,NaN,NaN,4.0,"Casa en barrrio cerrado en ""El Solar de Capilla"", lote de 40 por 60, casa tipo country con arboles frutales, muy buena sombra, pileta con filtro nuevo, galeria,con kichinette, agua fria caliente, parrilla, ventiladores de techo en galeria de 10 por 4. 3 dormitorios, 2 baños, gran living comedor,con hogar a leña, pasaplatos y cocina,gas, termo,direct tv,alarma,.Lote central, club house, canchas de tenis,2 piletas grandes y chicos,muy buena entrada desde la ruta,bajas expensas,vecinos q viven todo el año.-lugar muy agreste,muy LLEGAR:Por la autopista Panamericana, ramal Pilar hasta el cruce con la ruta 8. Se toma esta ruta hacia la derecha hasta el Km 69. Allí se debe girar a la derecha 700 metros y luego volver a girar a la derecha. El acceso se encuentra a 200 metros.La venta de este inmueble está sujeta a la tramitación del Código de Transferencia de Inmuebles (COTI), de conformidad con la normativa vigente (Res. AFIP 2371/08, 2439/08 y ccs.) por parte del propietario.",U$D 220.000 - Casa en Venta -,-36.0,"-36,-60",-60.0,Buenos Aires
5036,5426,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,235000.0,USD,4146457.5,235000.0,200.0,200.0,1175.000000,1175.000000,NaN,"CASA CUATRO AMBIENTES A LA VENTA EN CARDALESBARRIO ""CARDALES VILLAGE"" LOTE Nº 7CASA CUATRO AMBIENTES DE 200 MT2 CUBIERTOS SOBRE UN LOTE DE 600 MTS, EN DOS PLANTA AL FRENTE; CON RECEPCIÓN, LIVING AMPLIO, TOILETTE, COCINA SEPARADA CON AMOBLAMIENTO DE BAJO MESADA Y ALACENA, TRES DORMITORIOS (DORMITORIO PRINCIPAL EN SUITE), TRES BAÑOS COMPLETOS, PARQUE Y PILETA. VALOR U$S 235.000.- - - FEDERICO NEGRO ESTUDIO INMOBILIARIO",CASA EN VENTA,-36.0,"-36,-60",-60.0,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112918,117729,sell,house,Bs.As. G.B.A. Zona Norte,|Argentina|Bs.As. G.B.A. Zona Norte|,Argentina,Bs.As. G.B.A. Zona Norte,3435907.0,NaN,NaN,NaN,580000.0,USD,10233810.0,580000.0,423.0,418.0,1371.158392,1387.559809,NaN,"Excelente Casa en venta en Campana Centro. Zona del Arco. Consta de Planta Baja: frente amplio de ladrillo visto con retiro jardín frontal, portón doble automático,

In [42]:
#correjimos esos valores tomando los nombres de los partidos obtenidos desde el archivo de lat y long de geonames
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Norte', 'place_name'] = df['place_name_geonames']
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Oeste', 'place_name'] = df['place_name_geonames']
df.loc[df['place_name'] == 'Bs.As. G.B.A. Zona Sur', 'place_name'] = df['place_name_geonames']

In [43]:
print('Cantidad place_name con Bs.As. G.B.A. Zona Norte: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Norte'])))
print('Cantidad place_name con Bs.As. G.B.A. Zona Oeste: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Oeste'])))
print('Cantidad place_name con Bs.As. G.B.A. Zona Sur: ' + str(len(df[df['place_name'] == 'Bs.As. G.B.A. Zona Sur'])))
print('Cantidad place_name con Buenos Aires Costa Atlántica: ' + str(len(df[df['place_name'] == 'Buenos Aires Costa Atlántica'])))
print('Cantidad place_name con Buenos Aires Interior: ' + str(len(df[df['place_name'] == 'Buenos Aires Interior'])))

Cantidad place_name con Bs.As. G.B.A. Zona Norte: 0
Cantidad place_name con Bs.As. G.B.A. Zona Oeste: 0
Cantidad place_name con Bs.As. G.B.A. Zona Sur: 0
Cantidad place_name con Buenos Aires Costa Atlántica: 27
Cantidad place_name con Buenos Aires Interior: 106


In [44]:
#los datos de state_name ya están estandarizados
zonas = list(df['state_name'].unique())
zonas.sort()
zonas

['Bs.As. G.B.A. Zona Norte',
 'Bs.As. G.B.A. Zona Oeste',
 'Bs.As. G.B.A. Zona Sur',
 'Buenos Aires Costa Atlántica',
 'Buenos Aires Interior',
 'Capital Federal',
 'Catamarca',
 'Chaco',
 'Chubut',
 'Corrientes',
 'Córdoba',
 'Entre Ríos',
 'Formosa',
 'Jujuy',
 'La Pampa',
 'La Rioja',
 'Mendoza',
 'Misiones',
 'Neuquén',
 'Río Negro',
 'Salta',
 'San Juan',
 'San Luis',
 'Santa Cruz',
 'Santa Fe',
 'Santiago Del Estero',
 'Tierra Del Fuego',
 'Tucumán']

In [45]:
#los datos de state_name ya están estandarizados
zonas = list(df['place_name'].unique())
zonas.sort()
zonas

[' Country Maschwitz Club',
 ' los alamos',
 '9 de Abril',
 'Abasto',
 'Abril Club de Campo',
 'Acacias Blancas',
 'Acassuso',
 'Achiras',
 'Adolfo Alsina',
 'Adrogué',
 'Aeropuerto Internacional Ezeiza',
 'Agronomía',
 'Agua Blanca',
 'Agua de Oro',
 'Aguas Verdes',
 'Albanueva Barrio Cerrado',
 'Aldea Brasilera',
 'Aldea Salto',
 'Aldo Bonzi',
 'Alejandro Korn',
 'Alejandro Roca',
 'Alem',
 'Alfar',
 'Almafuerte',
 'Almagro',
 'Almirante Brown',
 'Alta Gracia',
 'Altamira',
 'Alto Los Cardales',
 'Altos de Hudson I',
 'Altos de Hudson II',
 'Altos de Manzanares 1 y 2',
 'Altos de Matheu',
 'Altos del Golf',
 'Altos del Pilar',
 'Aluminé',
 'Alvear',
 'Anisacate',
 'Apóstoles',
 'Arequito',
 'Armenia Country Club',
 'Arocena',
 'Arroyo Ceibal',
 'Arroyo Leyes',
 'Arroyo Seco',
 'Arturo Seguí',
 'Ascochinga',
 'Ataliva',
 'Avellaneda',
 'Azara',
 'Azul',
 'Bahía Blanca',
 'Bahía del Sol',
 'Balcarce',
 'Balnearia',
 'Balneario San Cayetano',
 'Balvanera',
 'Banda del Río Salí',
 'Banfi

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 26 columns):
Id                            116066 non-null int64
operation                     116066 non-null object
property_type                 116066 non-null object
place_name                    116066 non-null object
place_with_parent_names       116066 non-null object
country_name                  116066 non-null object
state_name                    116066 non-null object
geonames_id                   97893 non-null float64
lat-lon                       67784 non-null object
lat                           67784 non-null float64
lon                           67784 non-null float64
price                         98529 non-null float64
currency                      98528 non-null object
price_aprox_local_currency    98529 non-null float64
price_aprox_usd               98529 non-null float64
surface_total_in_m2           77706 non-null float64
surface_covered_in_m2         97229 non-null

In [47]:
#estandarizamos nombres de barrios de CABA (tomado del trabajo del grupo 2)
reemplazo_nombres = {'Abasto' : 'Balvanera',
                     'Barrio Norte' : 'Recoleta',
                     'Catalinas': 'Retiro',
                     'Congreso' : 'Balvanera',
                     'Las Cañitas' : 'Palermo',
                     'Once' : 'Balvanera',
                     'Pompeya' : 'Nueva Pompeya',
                     'Tribunales' : 'San Nicolás',
                     'Centro / Microcentro' : 'San Nicolás'}


df = df.replace({'place_name': reemplazo_nombres})      

# Datos de coordenadas de propiedades

#### Armamos 2 DataFrame:
    (A) 1 DataFrame para Ordenada Latitud: de columnas 'place_name' y 'lat'
    (B) 1 DataFrame para Ordenada Longitud: de columnas 'place_name' y 'lon'

(A) DF de Ordenada Latitud

In [48]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 

dflatitud=df[['place_name', 'lat']]
dflatitud.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Comprobamos cantidad de filas completas de la columna 'lat' en comparación al total de filas

In [49]:
dflatitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 2 columns):
place_name    116066 non-null object
lat           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.7+ MB


De (A) nos quedamos solo con los campos con datos, de la columna 'lat', desechando los valores 'NaN'

In [50]:
dflatitud_1=dflatitud.dropna()
dflatitud_1.head()

,place_name,lat
0,Mataderos,-34.661824
1,La Plata,-34.903883
2,Mataderos,-34.652262
3,Liniers,-34.647797
4,Centro,-38.002626


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lat' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [51]:
dflatitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67784 entries, 0 to 116063
Data columns (total 2 columns):
place_name    67784 non-null object
lat           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las latitudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Latitudes

In [52]:
dflatitud_1=dflatitud_1.groupby(['place_name']).mean()
dflatitud_1.head()

,lat
place_name,
Country Maschwitz Club,-34.377707
los alamos,-34.423989
9 de Abril,-34.756258
Abril Club de Campo,-34.814314
Acacias Blancas,-34.405774


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [53]:
len(dflatitud_1)

949

(B) DF de Ordenada Longitud

###### Armo un subset sólo con las columnas 'place_name' y 'lon'

In [54]:
# Definimos el subset del dataframe 'data' para quedarnos sólo con las columnas 'place_name' y 'lat'
# una vez definida se pasa el método ".head()" que despliega las primeras 5 filas del Dataframe 
dflongitud=df[['place_name', 'lon']]
dflongitud.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Comprobamos cantidad de filas completas de la columna 'lon' en comparación al total de filas

In [55]:
dflongitud.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116066 entries, 0 to 116065
Data columns (total 2 columns):
place_name    116066 non-null object
lon           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.7+ MB


De (B) nos quedamos solo con los campos con datos, de la columna 'lon', desechando los valores 'NaN'

In [56]:
dflongitud_1=dflongitud.dropna()
dflongitud_1.head()

,place_name,lon
0,Mataderos,-58.508839
1,La Plata,-57.964330
2,Mataderos,-58.522982
3,Liniers,-58.516424
4,Centro,-57.549447


Volvemos a tomar cantidad de datos por fila y vemos que se eliminaron los NaN del Dataframe.
Observamos también, que en 'lon' bajaron 23 filas más, lo que indica que en 'place_name' también eran NaN. Se eliminan son sólo 23 en 121220 (0,018%)

In [57]:
dflongitud_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67784 entries, 0 to 116063
Data columns (total 2 columns):
place_name    67784 non-null object
lon           67784 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.6+ MB


Calulamos la media de las longtudes por grupo de localidad, con el método ".groupby()" P.ej.: toma todas las filas de localidad igual a "Abasto", de la columna 'place_name' y calcula la media de las Longitudes

In [58]:
dflongitud_1=dflongitud_1.groupby(['place_name']).mean()
dflongitud_1.head()

,lon
place_name,
Country Maschwitz Club,-58.755411
los alamos,-58.601876
9 de Abril,-58.498288
Abril Club de Campo,-58.156066
Acacias Blancas,-58.760994


Comprobamos que la cantidad de filas vuelve a disminuir, puesto que sólo muestra la media de localidades únicas.

In [59]:
len(dflongitud_1)

949

Definimos un nuevo dataframe que una las columnas lat y lon de los puntos (A) y (B).
Para ello vamos utilizar el método ".merge()" de Pandas

In [60]:
dflatlon=pd.merge(dflatitud_1, dflongitud_1, left_on='place_name', right_on='place_name')
dflatlon

,lat,lon
place_name,,
Country Maschwitz Club,-34.377707,-58.755411
los alamos,-34.423989,-58.601876
9 de Abril,-34.756258,-58.498288
Abril Club de Campo,-34.814314,-58.156066
Acacias Blancas,-34.405774,-58.760994
...,...,...
William Morris,-34.581910,-58.655338
Yerba Buena,-26.813276,-65.295010
Zelaya,-35.053791,-59.660925


Tomamos al nuevo dataframe y generamos un nuevo índice para que 'place_name' vuelva a ser una variable del df

In [61]:
dflatlon.reset_index(inplace=True)
dflatlon

,place_name,lat,lon
0,Country Maschwitz Club,-34.377707,-58.755411
1,los alamos,-34.423989,-58.601876
2,9 de Abril,-34.756258,-58.498288
3,Abril Club de Campo,-34.814314,-58.156066
4,Acacias Blancas,-34.405774,-58.760994
...,...,...,...
944,William Morris,-34.581910,-58.655338
945,Yerba Buena,-26.813276,-65.295010
946,Zelaya,-35.053791,-59.660925
947,Zárate,-34.096436,-59.067763


Tomamos al dataframe original (df) y hacemos un Join con Pandas, sumando al final del DF las columnas con las medias de 'lat' y 'lon'

In [62]:
df = df.merge(dflatlon, how='left', left_on='place_name', right_on='place_name', suffixes=('','_mediaPorZona'))
#df.head(5)

##### Property_Type => OK

In [63]:
#Vemos los tipos de propiedades existentes, y su correspondiente cantidad

df['property_type'].value_counts()

apartment    66572
house        39758
PH           5664 
store        4072 
Name: property_type, dtype: int64

In [64]:
#Revisamos si alguna linea tiene el tipo de propiedad vacio
#Vemos que no, que todas las 121.220 lineas existentes tienen el campo completo

df['property_type'].value_counts().sum()

116066

In [65]:
house = df.loc[:, 'property_type'] == 'house'
print("La cantidad de house es: ",house.sum())

apartment = df.loc[:, 'property_type'] == 'apartment'
print("La cantidad de apartment es: ",apartment.sum())

PH = df.loc[:, 'property_type'] == 'PH'
print("La cantidad de PH es: ",PH.sum())

store = df.loc[:, 'property_type'] == 'store'
print("La cantidad de store es: ",store.sum())

tipodepropiedad = house & apartment & PH & store
print("La cantidad de propiedades sin tipo es: ",tipodepropiedad.sum())

La cantidad de house es:  39758
La cantidad de apartment es:  66572
La cantidad de PH es:  5664
La cantidad de store es:  4072
La cantidad de propiedades sin tipo es:  0


##### Surface (covered y total)

###### Opcion 1: funcion para obtener metros cuadrados (a partir de surface_total_in_m2 y surface_covered_in_m2)

In [66]:
print('Cantidad registros surface_covered_in_m2 sin valor: ' + str(df['surface_covered_in_m2'].isnull().sum()))
print('Cantidad registros surface_total_in_m2 sin valor: ' + str(df['surface_total_in_m2'].isnull().sum()))

Cantidad registros surface_covered_in_m2 sin valor: 18837
Cantidad registros surface_total_in_m2 sin valor: 38360


###### Vamos a tratar de llenar los datos faltantes de superficie total en el dataset, rellenando con el promedio por barrio de cada publicacion

In [67]:
#Primero vamos a ordenar el dataset para poder tener los partidos en orden y limpios
df.rename(index=str, inplace=True)

In [68]:
#Primero hacmemos data wrangling de la columna "place_with_parent_names"

df.place_with_parent_names = df.place_with_parent_names.map(str.lower) #llevo a minusculas para evitar duplicados

grouped_places = df.groupby(['place_with_parent_names']) #agrupo por place_with_parent_names

dictio_places = grouped_places.groups.keys() #genero diccionario de places

cantidad_places = len(dictio_places) #cuento la cantidad de places distintos

print("Cantidad de place_with_parent_names distintos: ",cantidad_places)

Cantidad de place_with_parent_names distintos:  1164


In [69]:
count_x_places = grouped_places.agg({"Id": "count"}) #agrupo y cuento

count_x_places = count_x_places.rename(index=str, columns={"Id": "cantidad"}) #renombro la columna por cantidad

count_x_places = count_x_places.sort_values(by=['cantidad'], ascending=False) #ordeno por cantidad descendente

In [70]:
len(count_x_places.query("cantidad > 50")) # places con mas de <n> registros. Solo consulta

258

In [71]:
list_places = [sub_places.split('|') for sub_places in count_x_places.index]

df_places = pd.DataFrame(list_places, 
                         index = count_x_places.index, 
                         columns =['none1','pais','provincia','partido','localidad','barrio','none2'])

df_places = df_places.drop(['none1', 'none2'], axis=1) # elimino none1 y none2
df_places = df_places.drop(['pais'], axis=1) # elimino pais ya que todos son argentina
df_places[df_places.barrio.notnull()] # el unico que tiene barrios es tigre-nordelta
df_places = df_places.drop(['barrio'], axis=1) # elimino barrio tambien

In [72]:
# Para cada columna busco vacios y asigno None

for column_name in df_places.columns:
    df_places[column_name][df_places[column_name].apply(lambda column_name: True if regex.search('^\s*$', str(column_name)) else False)]=None

df_places = df_places.sort_values(by=['provincia','partido','localidad']) #ordeno

# creo df de provincias partidos y localidades
df_provincias = pd.DataFrame(df_places.provincia.unique(),columns=['nombre'])
df_partidos = pd.DataFrame(df_places.partido.unique(),columns=['nombre'])
df_localidades = pd.DataFrame(df_places.localidad.unique(),columns=['nombre'])

def buscar_reemplazar_place_column(row,column_name,df_base):
    if row[column_name]:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_places.provincia =  df_places.apply(buscar_reemplazar_place_column,args=('provincia',df_provincias),axis=1)
df_places.partido =  df_places.apply(buscar_reemplazar_place_column,args=('partido',df_partidos),axis=1)
df_places.localidad =  df_places.apply(buscar_reemplazar_place_column,args=('localidad',df_localidades),axis=1)
df_places.sample(5)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  app.launch_new_instance()


,provincia,partido,localidad
place_with_parent_names,,,
|argentina|misiones|campo grande|,17,387.0,NaN
|argentina|buenos aires interior|san miguel del monte|,4,117.0,NaN
|argentina|córdoba|manfredi|,10,262.0,NaN
|argentina|capital federal|paternal|,5,160.0,NaN
|argentina|buenos aires costa atlántica|mar del plata|san juan|,3,50.0,226.0


In [73]:
# agrega la relacion para las columnas que se vayan pasando respecto al dataframe de provincias, localidades y partidos

def agregar_columna_place(row,column_name,test):
    if (row.place_with_parent_names):
        return df_places.loc[row.place_with_parent_names][column_name]
    
provincias = df.apply(agregar_columna_place,args=('provincia','random'),axis=1)
df['provincia'] = pd.Series(provincias, index=df.index)
partidos = df.apply(agregar_columna_place,args=('partido','random'),axis=1)
df['partido'] = pd.Series(partidos, index=df.index)
localidades = df.apply(agregar_columna_place,args=('localidad','random'),axis=1)
df['localidad'] = pd.Series(localidades, index=df.index)
df.sample(3)

,Id,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,description,title,lat_geonames,lat-lon_geonames,lon_geonames,place_name_geonames,lat_mediaPorZona,lon_mediaPorZona,provincia,partido,localidad
45112,46198,sell,apartment,Tigre,|argentina|bs.as. g.b.a. zona norte|tigre|tigre|,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4104642,-58.5889331",-34.410464,-58.588933,172272.0,USD,3039653.3,172272.0,98.0,45.0,1757.877551,3828.266667,2.0,"Dpto a Estrenar de 2 amb. en Complejo denominado"" María de la Ribera""de alta categoría a mts del Museo de Tigre con vista al río Luján y a los jardines del Museo en el contrafrente.Estar-Comedor , cocina , 1 dormitorios en suite con vestidor, toilette y balcón y terraza propia de 50 m2.-Pisos Porcellanatto.El edificio cuenta con terraza común con espectacular vista al delta, donde habrá un salón cerrado para eventos, sala de cine , laundry de uso común , Solarium, Office, Parrilla y toilettes.-Cochera incluidaMatriculado:Esteban CorazzaCMCPSI 5528Tel. +54 11 //",Paseo Victorica 800 3° G Depto. A ESTRENAR 2 ambt. Tigre Residencial,NaN,NaN,NaN,NaN,-34.416773,-58.613514,0.0,8.0,232.0
41202,42198,sell,apartment,Pilar,|argentina|bs.as. g.b.a. zona norte|pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,"-34.4338414475,-58.8320911417",-34.433841,-58.832091,126000.0,USD,2223207.0,126000.0,79.0,79.0,1594.936709,1594.936709,3.0,"Muy bien ubicado dentro del complejo. Primer piso por escalera.Planta baja: cocina integrada con barra desayunadora, lavadero incorporado. Living comedor con salida al balcon terraza con parrilla y excelente vista al bosque interno del complejo. Planta alta: dormitorio principal, baño completo con bañera, dormitorio secundario. Muy buen estado. Cochera descubierta.Amenities: SUM, Piscina con solarium. Seguridad hs En cumplimiento de la Ley 2340 CUCICBA, Ley 10.973 de la Prov.Bs.As., Ley Nacional 25.028, Ley 22.802 de Lealtad Comercial, Ley 24.240 de Defensa al Consumidor, las normas del Código Civil y Comercial de la Nación y Constitucionales, los agentes NO ejercen el corretaje inmobiliario. Todas las operaciones inmobiliarias son objeto de intermediación y conclusión por parte de los martilleros y corredores colegiados, cuyos datos se exhiben debajo del nombre de la inmobiliaria.",Duplex de ambientes en Venta en Apart del Pinazo,-34.41667,"-34.41667,-58.83333",-58.83333,Partido de Pilar,-34.445218,-58.878634,0.0,4.0,NaN
49270,50467,sell,PH,Mar del Plata,|argentina|buenos aires costa atlántica|mar del plata|,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-37.9989542,-57.5790904",-37.998954,-57.579090,35000.0,USD,617557.5,35000.0,35.0,30.0,1000.000000,1166.666667,2.0,"PH de 2 amb. en PB primero por pasillo, cocina incorporada, dorm. con placard, baño con ducha, patio con pileta de lavadero. Buen estado. Apto Crèdito.",PH 2 amb. PB p/pasillo Avellaneda y Av. Jara,-38.00042,"-38.00042,-57.5562",-57.55620,Mar del Plata,-38.006349,-57.553479,3.0,50.0,NaN


In [74]:
df['state_name'].value_counts()

Capital Federal                 30685
Bs.As. G.B.A. Zona Norte        24763
Bs.As. G.B.A. Zona Sur          13556
Córdoba                         11582
Buenos Aires Costa Atlántica    9911 
Bs.As. G.B.A. Zona Oeste        9048 
Santa Fe                        9033 
Buenos Aires Interior           2251 
Río Negro                       792  
Mendoza                         656  
Tucumán                         635  
Neuquén                         587  
Corrientes                      575  
Misiones                        453  
Entre Ríos                      365  
Salta                           276  
Chubut                          259  
San Luis                        245  
La Pampa                        153  
Chaco                           57   
San Juan                        40   
Formosa                         32   
Tierra Del Fuego                31   
Catamarca                       27   
Jujuy                           26   
Santa Cruz                      20   
Santiago Del

In [75]:
df['provincia'].value_counts()

5.0     30685
0.0     24763
2.0     13556
10.0    11582
3.0     9911 
1.0     9048 
24.0    9033 
4.0     2251 
19.0    792  
16.0    656  
27.0    635  
18.0    587  
9.0     575  
17.0    453  
11.0    365  
20.0    276  
8.0     259  
22.0    245  
14.0    153  
7.0     57   
21.0    40   
12.0    32   
26.0    31   
6.0     27   
13.0    26   
23.0    20   
25.0    4    
15.0    4    
Name: provincia, dtype: int64

In [76]:
# Analisis del cambio de columnas realizado con place_with parent names

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)
df_place_name = df.place_name

df_place_name_not_in = df_place_name[~(df_place_name.isin(df_partidos.nombre))]
df_place_name_not_in_loc = df_place_name_not_in[~(df_place_name_not_in.isin(df_localidades.nombre))]
df_place_name_not_in_loc_prov = df_place_name_not_in_loc[~(df_place_name_not_in_loc.isin(df_provincias.nombre))]
df_place_name_not_in_loc_prov.unique()
#Puedo eliminar la columna place_name

array(['barrio el golf', 'buenos aires', 'barrio las glorietas',
       'nueva pompeya', 'barrio los lagos', 'enyoi', 'qbay yacht',
       'barrio la alameda', 'islas del canal', 'barrio los alisos',
       'barrioportezuelo', 'barrio la isla', 'barrio los tilos',
       'barrio los sauces', 'barrio cabos del lago',
       'barrio los castores', 'barrio barrancas del lago',
       'barrio el yacht'], dtype=object)

In [77]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en partidos

df.place_name = df.place_name[df.place_name.notnull()].map(str.lower)

def buscar_reemplazar_place_definidos(row,column_name,df_base): # funcion que buscar y reemplaza de la columna base (provincia, localidad, partido)
    a = df_base.nombre.str.contains(row[column_name], regex=False).any()
    if a:
        idx = df_base.index[df_base.nombre == row[column_name]]
        return int(idx.data[0])
    
df_place_name = df.place_name
mask_in_partidos = (df_place_name.isin(df_partidos.nombre))
mask_not_column_partido = df.partido.isnull()

print("Partidos con null:" + str(df.partido.isnull().sum()))

Partidos con null:4403


In [78]:
df_reemplazar_part = df[mask_not_column_partido&mask_in_partidos].apply(buscar_reemplazar_place_definidos,args=('place_name',df_partidos),axis=1)
df.partido.update(df_reemplazar_part)
print("Partidos con null luego de procesar:" + str(df.partido.isnull().sum()))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Partidos con null luego de procesar:1686


In [79]:
# Reemplazo los valores que encuentro en place_name y que estan definidos en localidades

df_place_name = df.place_name
mask_in_localidades = (df_place_name.isin(df_localidades.nombre))
mask_not_column_localidad = df.localidad.isnull()

print("Localidades con null: " + str(df.localidad.isna().sum()))

df_reemplazar_loc = df[mask_not_column_localidad&mask_in_localidades].apply(buscar_reemplazar_place_definidos,args=('place_name',df_localidades),axis=1)
df.localidad.update(df_reemplazar_loc)

print("Localidades con null luego de procesar: " + str(df.localidad.isna().sum()))

Localidades con null: 76915


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  if __name__ == '__main__':


Localidades con null luego de procesar: 65576


###### Ahora vamos a limpiar la columna surface

In [80]:
df_title = df[df.title.notnull()]
df_title.title = df_title.title.map(str.lower)
pattern_m2 = regex.compile("(\d+\s*) m2")

def get_m2(row):
    result = pattern_m2.search(row)
    try:
        str_aux = result.group(1)
        array_m2 = str_aux.split()
        m2 = array_m2[-1]
        try:
            m2 = float(m2)
            return m2
        except:
            return np.nan;
    except:
        return np.nan

print("Cantidad de nulos m2 total surface: ",df.surface_total_in_m2.isnull().sum())

m2_from_title = df_title.title.apply(get_m2)
m2_from_title[m2_from_title.notnull()]
df.surface_total_in_m2.update(m2_from_title)

print("Cantidad de nulos luego de procesar: ",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos m2 total surface:  38360
Cantidad de nulos luego de procesar:  37906


In [81]:
#Vamos a generar los 39.328 datos faltantes en surface_total en la tabla, y lo vamos a hacer
#rellenando con el promedio del barrio de cada propiedad

print("Cantidad de nulos en surface_total_in_m2 antes:",df.surface_total_in_m2.isnull().sum())

# creamos una proporción de metros cubiertos sobre metros totales - no puede ser mayor a 1!
propcubierto = df.surface_covered_in_m2 / df.surface_total_in_m2
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

Cantidad de nulos en surface_total_in_m2 antes: 37906


In [82]:
#Relevamos algunos casos anómalos, donde la proporción da mayor a uno (inverosímil). Desestimamos estos 1200 registros para este procedimiento.
tempmask = propcubierto > 1
np.sum(tempmask)

1194

In [83]:
#Creamos una nueva variable sin esos datos anómalos
mask = propcubierto < 1
propcubierto_clean  = propcubierto[mask]

In [84]:
#Hay dos casos anómalos donde superficie cubierta es cero, los reemplazamos por np.nan para evitar conflictos en el siguiente paso.
masksurface0 = (df.surface_covered_in_m2 == 0)
df.surface_covered_in_m2[masksurface0].fillna(np.nan)

#Agregamos la columna al dataframe
df['propcubierto']=propcubierto_clean

#Agrupamos por provincia(partido) el porcentaje promedio de m2cubierto/m2total
avg_propcubiertobarrio = df.groupby('partido')["propcubierto"].mean().sort_values(ascending = False)

#Cantidad de datos para calcular la proporcion 
avg_propcubiertobarriocount = df.groupby('partido')["propcubierto"].count().sort_values(ascending = False)

#Condición de la regla 1
removerporcantidadmask = avg_propcubiertobarriocount > 30

#Cantidad de datos existentes en tabla
datos_en_tabla = df.groupby('partido')['partido'].count().sort_values(ascending = False)

#Divido los datos existentes sobre los datos totales y obtengo la relación para la regla DOS
proporcion = avg_propcubiertobarriocount / datos_en_tabla
proporcion.round(2).sort_values(ascending = True)

#Condición de la regla 2
removerporproporcionmask = proporcion > 0.25

#Genero máscara con ambas condiciones
proporcionmask2 = removerporcantidadmask & removerporproporcionmask
propvalidados = avg_propcubiertobarrio[proporcionmask2]

#Ahora iteramos por las propiedades que tienen el dato faltante de superficie total y les inputamos la proporción promedio
#del barrio al que pertenecen, usando como dato la superficie cubierta.
# SUPERFICIE TOTAL = (SUPERFICIE CUBIERTA / PROPORCION CUBIERTO TOTAL)

surface_total_in_m2_clean = []
for index, row in df.iterrows():
    
    if pd.isnull(row.surface_total_in_m2):    
        if(row.partido in propvalidados.index):        
            surface_total_in_m2_clean.append(row.surface_covered_in_m2 / propvalidados.loc[row.partido]) #VERSION CON PROPVALIDADOS NO CORRE
        else:
            surface_total_in_m2_clean.append(row.surface_total_in_m2)    
    else:
        surface_total_in_m2_clean.append(row.surface_total_in_m2)
df["surface_total_in_m2_nueva"] = surface_total_in_m2_clean
df["surface_total_in_m2_nueva"]

#dropeamos los casos NaN
surfacetotalnueva_condatos = df.surface_total_in_m2_nueva.dropna()

#y finalmente, los reemplazamos en la columna "surface_total_in_m2_nueva" original del DF
df.surface_total_in_m2.update(surfacetotalnueva_condatos)

print("Cantidad de nulos en surface_total_in_m2 despues:",df.surface_total_in_m2.isnull().sum())

Cantidad de nulos en surface_total_in_m2 despues: 20504


In [85]:
#eliminamos columnas auxiliares generadas
df = df.drop(labels=['provincia', 'partido', 'localidad', 'propcubierto', 'surface_total_in_m2_nueva'], axis= 1)

In [86]:
df.columns

Index(['Id', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms',
       'description', 'title', 'lat_geonames', 'lat-lon_geonames',
       'lon_geonames', 'place_name_geonames', 'lat_mediaPorZona',
       'lon_mediaPorZona'],
      dtype='object')

In [87]:
df.surface_total_in_m2.isnull().sum()

20504

### Intentamos obtener las superficies faltantes a partir de las medias de las mismas zonas

In [88]:
#generamos un df auxiliar para trabajar la obtencion de los precios que faltan
df_aux = df

In [89]:
print('Valor maximo: ' + str(df_aux['surface_total_in_m2'].max()))
print('Valor minimo: ' + str(df_aux['surface_total_in_m2'].min()))

Valor maximo: 217550.20281411518
Valor minimo: 0.0


In [90]:
df_aux['surface_total_in_m2'].describe(percentiles=[.1, .25, .5, .6, .7, .75, .95]).apply(lambda x: format(x, 'f'))

count    95562.000000 
mean     230.234362   
std      1712.233990  
min      0.000000     
10%      38.000000    
25%      52.000000    
50%      89.000000    
60%      120.000000   
70%      174.000000   
75%      208.264724   
95%      695.845109   
max      217550.202814
Name: surface_total_in_m2, dtype: object

In [91]:
#para obtener muestras utilizables, eliminamos valores que estan por debajo del precentil 25% y por arriba del percentil 70%
df_aux = df_aux[df_aux['surface_total_in_m2'] >= 52]
df_aux = df_aux[df_aux['surface_total_in_m2'] <= 174]

In [92]:
df_aux['surface_total_in_m2'].describe(percentiles=[.1, .25, .5, .75, .95]).apply(lambda x: format(x, 'f'))

count    43036.000000
mean     92.301302   
std      32.973239   
min      52.000000   
10%      56.000000   
25%      65.000000   
50%      83.000000   
75%      115.000000  
95%      159.079592  
max      174.000000  
Name: surface_total_in_m2, dtype: object

In [93]:
primerCuartil = df_aux['surface_total_in_m2'].quantile(0.25)

In [94]:
tercerCuartil = df_aux['surface_total_in_m2'].quantile(0.75)

In [95]:
rangoInterCuartil = tercerCuartil - primerCuartil

In [96]:
limiteInferior = primerCuartil - (rangoInterCuartil * 1.5)

In [97]:
limiteSuperior = tercerCuartil + (rangoInterCuartil * 1.5)

In [98]:
print('Primer cuartil: ' + str(primerCuartil))
print('Tercer cuartil: ' + str(tercerCuartil))
print('Rando intercuartil: ' + str(rangoInterCuartil))
print('Limite outlier inferior: ' + str(limiteInferior))
print('Limite outlier superior: ' + str(limiteSuperior))

Primer cuartil: 65.0
Tercer cuartil: 115.0
Rando intercuartil: 50.0
Limite outlier inferior: -10.0
Limite outlier superior: 190.0


In [99]:
#definimos topes para sacar los outliers en USD
outlier_min_sup_m2 = 0
outlier_max_sup_m2 = 190

In [100]:
#definimos un data frame auxiliar para el proceso
df_sup_m2_nan_outliers = df

In [101]:
def changeSupM2OutliersToNan(dataRow):
    #si el dato es un outlier lo pasamos a Nan, para que despues en el calculo de la media para el agrupamiento no afecte    
    if(not math.isnan(dataRow.surface_total_in_m2) and (dataRow.surface_total_in_m2 < outlier_min_sup_m2 or dataRow.surface_total_in_m2 > outlier_max_sup_m2)):
        dataRow.surface_total_in_m2 = np.nan
    return dataRow

In [102]:
#en el df auxiliar, seteamos todos las superficies por m2 outliers a NAN
df_sup_m2_nan_outliers = df_sup_m2_nan_outliers.apply(changeSupM2OutliersToNan, axis=1)

In [103]:
print('Cantidad de outliers: ' + str(len(df_sup_m2_nan_outliers[df_sup_m2_nan_outliers['surface_total_in_m2'].isnull()])))
print('Valor maximo sacando outliers: ' + str(df_sup_m2_nan_outliers['surface_total_in_m2'].max()))
print('Valor minimo sacando outliers: ' + str(df_sup_m2_nan_outliers['surface_total_in_m2'].min()))

Cantidad de outliers: 46828
Valor maximo sacando outliers: 190.0
Valor minimo sacando outliers: 0.0


In [104]:
#agrupamos para obtener la media de la superficie total en m2 por propiedades categóricas
sup_m2_grouped = df_sup_m2_nan_outliers.groupby(by=['operation', 'property_type', 'state_name', 'place_name'])

In [105]:
df['sup_total_m2_estandarizada_por_grupo'] = sup_m2_grouped['surface_total_in_m2'].transform(lambda grp: np.mean(grp))

In [106]:
def fillEmptyTotalSurfaceM2FromMeanTotalSurfaceM2(dataRow):
    if(math.isnan(dataRow.surface_total_in_m2) or (not math.isnan(dataRow.surface_total_in_m2) and dataRow.surface_total_in_m2 < outlier_min_sup_m2)\
      or (not math.isnan(dataRow.surface_total_in_m2) and dataRow.surface_total_in_m2 > outlier_max_sup_m2)):        
        dataRow.surface_total_in_m2 = dataRow.sup_total_m2_estandarizada_por_grupo
    return dataRow

In [107]:
df = df.apply(fillEmptyTotalSurfaceM2FromMeanTotalSurfaceM2, axis=1)

In [108]:
print('Cantidad de sup total m2 sin valor: ' + str(len(df[df['surface_total_in_m2'].isnull()])))

Cantidad de sup total m2 sin valor: 2616


In [109]:
#Aun quedan valores de sup total m2 sin completar, son valores que son unicos por place_name, realizamos de vuelta el procedimiento
#pero esta vez agrupamos excluyendo el nivel de localidad, lo hacemos hasta provincia para obtener un valor mas general y
#completar los faltantes
#agrupamos para obtener la media de la superficie total en m2 por propiedades categóricas
sup_m2_grouped_without_place_name = df_sup_m2_nan_outliers.groupby(by=['operation', 'property_type', 'state_name'])

In [110]:
df['sup_total_m2_estandarizada_por_grupo'] = sup_m2_grouped_without_place_name['surface_total_in_m2'].transform(
    lambda grp: np.mean(grp))

In [111]:
df = df.apply(fillEmptyTotalSurfaceM2FromMeanTotalSurfaceM2, axis=1)

In [112]:
print('Cantidad de prices sin valor luego de completar con valores mas generales: ' + str(len(df[df['price'].isnull()])))

Cantidad de prices sin valor luego de completar con valores mas generales: 17537


In [113]:
#eliminamos las columnas auxiliares para completar price del data frame
df = df.drop(labels=['sup_total_m2_estandarizada_por_grupo'], axis = 1)

# Precio

### Todos los valores del precio que estén en ARS los pasamos a USD, tomando como valor válido para esto el price_aprox_usd, para empezar a llevar todos los precios a USD

In [114]:
def changeARSPriceForPriceAproxUSD(dataRow):
    if(not math.isnan(dataRow.price_aprox_usd)):
        dataRow.price = dataRow.price_aprox_usd
        dataRow.currency = 'USD'
    return dataRow

In [115]:
#Convertimos los price existentes de ARS a USD, donde existe el price_aprox_usd
df = df.apply(changeARSPriceForPriceAproxUSD, axis=1)

In [116]:
#Por el momento tenemos solamente currency válidas en USD, a continuación vamos a extraer precios desde description y title
#quizás ahí obtengamos nuevos valores en ARS
df['currency'].unique()

array(['USD', nan], dtype=object)

### Obtenemos valores desde las columnas description y title

In [117]:
len(df[df['price'].isnull()])

17537

In [118]:
def completarValoresFaltantesEnFilaPrice(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar analize_description_title.ipynb
    updatedDataFameRow = updatePriceFromRowData(dataFrameRow)
        
    return updatedDataFameRow


In [119]:
df = df.apply(completarValoresFaltantesEnFilaPrice, axis=1);

In [120]:
len(df[df['price'].isnull()])

13647

In [121]:
#vemos que volvemos a tener datos con monedas en ARS
df['currency'].unique()

array(['USD', nan, 'ARS'], dtype=object)

### Correccion de valores de precios, pasamos todos los precios a USD. Sacar los outliers. Luego calcular la media del precio en USD por barrio. Finalmente con esas medias, actualizar los valores de las filas que no tengan precio y tambien aquellos que tienen outliers

In [122]:
#eliminamos filas que tienen precios en monedas que no son ARS ni USD
filasConMonedasNoUtilizadas = df[df['currency'] == 'UYU'].index

#borramos las filas con los correspondientes indices
df = df.drop(filasConMonedasNoUtilizadas)

#eliminamos filas que tienen precios en monedas que no son ARS ni USD
filasConMonedasNoUtilizadas = df[df['currency'] == 'PEN'].index

#borramos las filas con los correspondientes indices
df = df.drop(filasConMonedasNoUtilizadas)

#reseteamos los índices para poder seguir usándolos sin problemas
df = df.reset_index(drop=True);


In [123]:
#método que transforma el dato price desde cualquier moneda a USD
def setStandardUSDPrice(dataRow):
    if(not math.isnan(dataRow.price) and dataRow.currency != 'USD'):
        #en caso de que la fila no tenga currency, se asume que tiene USD
        moneda = 'USD'
        
        if(not pd.isnull(dataRow.currency) and dataRow.currency != ''):
            moneda = dataRow.currency
        
        dataRow.price = dataRow.price / conversion_USD_a_monedas_locales[moneda]
        dataRow.currency = 'USD'
           
    return dataRow

In [124]:
#Convertimos todos los price existentes a USD
df = df.apply(setStandardUSDPrice, axis=1)

In [125]:
#vemos que volvemos a tener solamente USD, y los Nan
df['currency'].unique()

array(['USD', nan], dtype=object)

In [126]:
#generamos un df auxiliar para trabajar la obtencion de los precios que faltan
df_aux = df

In [127]:
print('Valor maximo: ' + str(df_aux['price'].max()))
print('Valor minimo: ' + str(df_aux['price'].min()))

Valor maximo: 123456789.0
Valor minimo: 0.0


In [128]:
df_aux['price'].describe(percentiles=[.1, .25, .5, .6, .75, .95]).apply(lambda x: format(x, 'f'))

count    102419.000000   
mean     237378.643719   
std      548513.619546   
min      0.000000        
10%      61522.456000    
25%      88000.000000    
50%      142558.000000   
60%      175000.000000   
75%      260000.000000   
95%      695000.000000   
max      123456789.000000
Name: price, dtype: object

In [129]:
#para obtener muestras utilizables, eliminamos valores que estan por debajo del precentil 25% y por arriba del percentil 
df_aux = df_aux[df_aux['price'] >= 88000]
df_aux = df_aux[df_aux['price'] <= 260000]

In [130]:
df_aux['price'].describe(percentiles=[.1, .25, .5, .75, .95]).apply(lambda x: format(x, 'f'))

count    51324.000000 
mean     153277.601968
std      48158.282141 
min      88000.000000 
10%      96000.000000 
25%      115000.000000
50%      142083.420000
75%      185000.000000
95%      249000.000000
max      260000.000000
Name: price, dtype: object

In [131]:
primerCuartil = df_aux['price'].quantile(0.25)

In [132]:
tercerCuartil = df_aux['price'].quantile(0.75)

In [133]:
rangoInterCuartil = tercerCuartil - primerCuartil

In [134]:
limiteInferior = primerCuartil - (rangoInterCuartil * 1.5)

In [135]:
limiteSuperior = tercerCuartil + (rangoInterCuartil * 1.5)

In [136]:
print('Primer cuartil: ' + str(primerCuartil))
print('Tercer cuartil: ' + str(tercerCuartil))
print('Rando intercuartil: ' + str(rangoInterCuartil))
print('Limite outlier inferior: ' + str(limiteInferior))
print('Limite outlier superior: ' + str(limiteSuperior))

Primer cuartil: 115000.0
Tercer cuartil: 185000.0
Rando intercuartil: 70000.0
Limite outlier inferior: 10000.0
Limite outlier superior: 290000.0


In [137]:
print('Valor maximo: ' + str(df['price'].max()))
print('Valor minimo: ' + str(df['price'].min()))

Valor maximo: 123456789.0
Valor minimo: 0.0


In [138]:
#definimos topes para sacar los outliers en USD
outlier_min_price = 10000
outlier_max_price = 290000

In [139]:
#definimos un data frame auxiliar para el proceso
df_usd_prices_nan_outliers = df

In [140]:
def changeStandarUSDPriceOutliersToNan(dataRow):
    #si el dato es un outlier lo pasamos a Nan, para que despues en el calculo de la media para el agrupamiento no afecte
    #pero seguimos teniendo la fila para actualizarle el valor del precio
    if(not math.isnan(dataRow.price) and (dataRow.price < outlier_min_price or dataRow.price > outlier_max_price)):
        dataRow.price = np.nan
    return dataRow

In [141]:
#en el df auxiliar, seteamos todos los precios outliers a NAN
df_usd_prices_nan_outliers = df_usd_prices_nan_outliers.apply(changeStandarUSDPriceOutliersToNan, axis=1)

In [142]:
print('Cantidad de outliers: ' + str(len(df_usd_prices_nan_outliers[df_usd_prices_nan_outliers['price'].isnull()])))
print('Valor maximo sacando outliers: ' + str(df_usd_prices_nan_outliers['price'].max()))
print('Valor minimo sacando outliers: ' + str(df_usd_prices_nan_outliers['price'].min()))

Cantidad de outliers: 37217
Valor maximo sacando outliers: 290000.0
Valor minimo sacando outliers: 10000.0


In [143]:
#agrupamos para obtener la media del precio en USD por propiedades categóricas
usd_price_grouped = df_usd_prices_nan_outliers.groupby(by=['operation', 'property_type', 'state_name', 'place_name'])

In [144]:
df['precio_estandarizado_usd_por_grupo'] = usd_price_grouped['price'].transform(lambda grp: np.mean(grp))

In [145]:
def fillEmptyPricesFromMeanUSDPrice(dataRow):
    if(math.isnan(dataRow.price) or (not math.isnan(dataRow.price) and dataRow.price < outlier_min_price)\
      or (not math.isnan(dataRow.price) and dataRow.price > outlier_max_price)):
        dataRow.currency= 'USD'
        dataRow.price = dataRow.precio_estandarizado_usd_por_grupo
    return dataRow

In [146]:
df = df.apply(fillEmptyPricesFromMeanUSDPrice, axis=1)

In [147]:
print('Cantidad de prices sin valor: ' + str(len(df[df['price'].isnull()])))

Cantidad de prices sin valor: 1337


In [148]:
#Aun quedan valores de precios sin completar, son valores que son unicos por place_name, realizamos de vuelta el procedimiento
#pero esta vez agrupamos excluyendo el nivel de localidad, lo hacemos hasta provincia para obtener un valor mas general y
#completar los faltantes
#agrupamos para obtener la media del precio en USD por propiedades categóricas
usd_price_grouped_without_place_name = df_usd_prices_nan_outliers.groupby(by=['operation', 'property_type', 'state_name'])

In [149]:
df['precio_estandarizado_usd_por_grupo'] = usd_price_grouped_without_place_name['price'].transform(
    lambda grp: np.mean(grp))

In [150]:
df = df.apply(fillEmptyPricesFromMeanUSDPrice, axis=1)

In [151]:
print('Cantidad de prices sin valor luego de completar con valores mas generales: ' + str(len(df[df['price'].isnull()])))

Cantidad de prices sin valor luego de completar con valores mas generales: 51


In [152]:
#Generalizamos aun mas el agrupamiento, sacando la provincia
usd_price_grouped_without_gral = df_usd_prices_nan_outliers.groupby(by=['operation', 'property_type'])

In [153]:
df['precio_estandarizado_usd_por_grupo'] = usd_price_grouped_without_gral['price'].transform(
    lambda grp: np.mean(grp))

In [154]:
df = df.apply(fillEmptyPricesFromMeanUSDPrice, axis=1)

In [155]:
print('Cantidad de prices sin valor luego de completar con valores mas generales: ' + str(len(df[df['price'].isnull()])))

Cantidad de prices sin valor luego de completar con valores mas generales: 0


In [156]:
#eliminamos las columnas auxiliares para completar price del data frame
df = df.drop(labels=['precio_estandarizado_usd_por_grupo'], axis = 1)

# Completar el valor del campo rooms

In [157]:
def completarValoresFaltantesEnFila(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms utilizando el método definido en la notebbok auxiliar analize_description_title.ipynb
    updatedDataFameRow = updateRoomsFromRowData(dataFrameRow)
        
    return updatedDataFameRow

In [158]:
df = df.apply(completarValoresFaltantesEnFila, axis=1);

# Datos en las columnas description y title

In [159]:
print('Cantidad de registros description con valor nulo: ' + str(df.description.isna().sum()))
print('Cantidad de registros title con valor nulo: ' + str(df.title.isna().sum()))

Cantidad de registros description con valor nulo: 0
Cantidad de registros title con valor nulo: 0


# Columna price_usd_per_m2

In [160]:
#funcion que recibe una fila del data frame y un diccionario con las conversiones entre monedas locales y el USD
#este metodo intenta calcular el precio por m2 en USD a partir de otras columnas del data frame
def updatePriceUSDPerM2(dataFrameRow, dolarConversion):
    
    conversion = 1
    #si el dato currency no tiene valor, asumimos que los valores son en USD (conversion = 1)
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency) and dataFrameRow.currency != ''):
        conversion = dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_usd_per_m2)):
        #precio por m2 USD= precio total USD / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
        
        #precio por m2 USD = (precio total $ * conversion moneda) / superficie total        
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
    
        #precio por m2 USD = precio por m2 * conversion moneda
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_per_m2)):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_per_m2 * conversion
            
        #precio por m2 USD = precio aprox por m2 USD / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_usd / dataFrameRow.surface_total_in_m2
            
        #precio por m2 USD = precio aprox por m2 local currency * conversion moenda / superficie total
        if(math.isnan(dataFrameRow.price_usd_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price_aprox_local_currency * conversion) / dataFrameRow.surface_total_in_m2
            
    return dataFrameRow

In [161]:
#contamos price per m2 en USD antes de la ejecucion
df['price_usd_per_m2'].isna().sum()

49265

In [162]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePriceUSDPerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [163]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_usd_per_m2'].isna().sum()

401

###### Columna price_per_m2

In [164]:
#similar al calculo del m2 por USD, el calculo del m2 en moneda local usa otras columnas del data frame para obtener el valor
#recibe tambien un diccionario con la conversion de monedas locales a USD
def updatePricePerM2(dataFrameRow, dolarConversion):
            
    #si el dato currency no tiene valor, asumimos que el valor precio por m2 a guardar es en ARS
    conversion = 1 / dolarConversion['ARS'] #si el dolar esta a 65 ARS => el factor de conversión de USD a ARS es 1 / 65
    
    #si currency tiene un valor, obtenemos el valor de conversion desde el diccionario dolarConversion
    if(not pd.isnull(dataFrameRow.currency)):
        conversion = 1 / dolarConversion[dataFrameRow.currency]
    
    #completamos solamente si la columna no tiene valor
    if(math.isnan(dataFrameRow.price_per_m2)):
        #precio por m2 = precio aprox total local currency / superficie total
        if(not math.isnan(dataFrameRow.price_aprox_local_currency) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = dataFrameRow.price_aprox_local_currency / dataFrameRow.surface_total_in_m2
            
        #precio por m2 = precio / superficie total
        if(not math.isnan(dataFrameRow.price) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_usd_per_m2 = (dataFrameRow.price * conversion) / dataFrameRow.surface_total_in_m2
        
        #precio por m2 = precio por m2 USD / conversion moneda
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_usd_per_m2)):
            dataFrameRow.price_per_m2 = dataFrameRow.price_usd_per_m2 / conversion
            
        #precio por m2 USD = (precio aprox por m2 USD / conversion) / superficie total
        if(math.isnan(dataFrameRow.price_per_m2) 
           and not math.isnan(dataFrameRow.price_aprox_usd) and not math.isnan(dataFrameRow.surface_total_in_m2) and dataFrameRow.surface_total_in_m2 != 0):
            dataFrameRow.price_per_m2 = (dataFrameRow.price_aprox_usd / conversion) / dataFrameRow.surface_total_in_m2
             
    return dataFrameRow

In [165]:
#contamos price per m2 en USD antes de la ejecucion
df['price_per_m2'].isna().sum()

30421

In [166]:
#aplicamos la funcion que actualiza los precios por m2 al data frame
df = df.apply(updatePricePerM2, axis=1, dolarConversion=conversion_USD_a_monedas_locales);

In [167]:
#verificamos la cantidad de vacíos luego de la ejecución
df['price_per_m2'].isna().sum()

401

# Agregamos algunos datos de amenities analizando las columnas description y title

##### Los métodos que aplican las regex sobre las columnas description y title están definidos en la notebook auxiliar analize_description_title

In [168]:
df['pileta'] = df.apply(lambda row: containsPool(row), axis = 1)
df['pileta'].value_counts()

0    101629
1    14437 
Name: pileta, dtype: int64

In [169]:
df['cochera'] = df.apply(lambda row: containsParking(row), axis = 1)
df['cochera'].value_counts()

0    74781
1    41285
Name: cochera, dtype: int64

In [170]:
df['balcon'] = df.apply(lambda row: containsBalcony(row), axis = 1)
df['balcon'].value_counts()

0    88648
1    27418
Name: balcon, dtype: int64

In [171]:
df['terraza'] = df.apply(lambda row: containsTerrace(row), axis = 1)
df['terraza'].value_counts()

0    89695
1    26371
Name: terraza, dtype: int64

In [172]:
df['parrilla'] = df.apply(lambda row: containsGrill(row), axis = 1)
df['parrilla'].value_counts()

0    100109
1    15957 
Name: parrilla, dtype: int64

# Ajustes de valores de las columnas

### En los casos en que la superficie cubierta sea mayor a la superficie total, se cambia el valor de la superficie cubierta por el de la total

In [173]:
# casos en los que sup cubierta > sup total
len(df[df['surface_total_in_m2'] < df['surface_covered_in_m2']])

27520

In [174]:
def ajustarValoresSuperficie(dataRow):
    if(not math.isnan(dataRow.surface_total_in_m2) and not math.isnan(dataRow.surface_covered_in_m2)):
        if(dataRow.surface_covered_in_m2 > dataRow.surface_total_in_m2):
            dataRow.surface_covered_in_m2 = dataRow.surface_total_in_m2
    return dataRow

In [175]:
df = df.apply(ajustarValoresSuperficie, axis=1)

In [176]:
# resultado luego de la correccion
len(df[df['surface_total_in_m2'] < df['surface_covered_in_m2']])

0

### En los casos de que la superficie total sea mayor a la cubierta, se agrega una nueva columna con el valor de la superficie no cubierta, que será la diferencia entre la superficie total y la superficie cubierta

In [177]:
# casos en la que la superficie cubierta no es el total de la superficie
len(df[df['surface_total_in_m2'] > df['surface_covered_in_m2']])

50909

In [178]:
df['surface_not_covered_in_m2'] = df['surface_total_in_m2'] - df['surface_covered_in_m2']

In [179]:
def updateNotCoveredSurface(dataFrameRow):
    if(not math.isnan(dataFrameRow.surface_total_in_m2) and not math.isnan(dataFrameRow.surface_covered_in_m2)):
        dataFrameRow.surface_not_covered_in_m2 = dataFrameRow.surface_total_in_m2 - dataFrameRow.surface_covered_in_m2
    return dataFrameRow

In [180]:
df = df.apply(updateNotCoveredSurface, axis=1)

In [181]:
# resultado luego de la correccion 
len(df[df['surface_total_in_m2'] == (df['surface_covered_in_m2'] + df['surface_not_covered_in_m2'])])

97187

### En los casos que tenga superficie cubierta pero no superficie total, se completa el valor de la total con la cubierta

In [182]:
def updateSurfaceTotalFromCoveredSurface(dataFrameRow):
    if(math.isnan(dataFrameRow.surface_total_in_m2) and not math.isnan(dataFrameRow.surface_covered_in_m2)):
        dataFrameRow.surface_total_in_m2 = dataFrameRow.surface_covered_in_m2
    return dataFrameRow
        

In [183]:
df = df.apply(updateSurfaceTotalFromCoveredSurface, axis=1)

### En los registros que tengan superficie total, pero no tengan superficie cubierta ni superficie no cubierta, se copiará el valor de la superficie total a la superficie cubierta. La superficie no cubierta será 0

In [184]:
def updateCoveredSurfaceFromTotalSurface(dataFrameRow):
    if(not math.isnan(dataFrameRow.surface_total_in_m2) and math.isnan(dataFrameRow.surface_covered_in_m2)):
        dataFrameRow.surface_covered_in_m2 = dataFrameRow.surface_total_in_m2
        dataFrameRow.surface_not_covered_in_m2 = 0
    return dataFrameRow

In [185]:
df = df.apply(updateCoveredSurfaceFromTotalSurface, axis=1)

In [186]:
#Estado final del data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116066 entries, 0 to 116065
Data columns (total 34 columns):
Id                            116066 non-null int64
operation                     116066 non-null object
property_type                 116066 non-null object
place_name                    116066 non-null object
place_with_parent_names       116066 non-null object
country_name                  116066 non-null object
state_name                    116066 non-null object
geonames_id                   97893 non-null float64
lat-lon                       67784 non-null object
lat                           67784 non-null float64
lon                           67784 non-null float64
price                         116066 non-null float64
currency                      116066 non-null object
price_aprox_local_currency    98529 non-null float64
price_aprox_usd               98529 non-null float64
surface_total_in_m2           116045 non-null float64
surface_covered_in_m2         116045 non-